In [1]:
import os
os.chdir('../')

## Observe variation in the observation space

In [2]:
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
    
config = None
with open('config.json') as f:
    config = json.load(f)


In [3]:
import gym

all_observations = []

In [4]:

env = gym.make('CartPole-v1')

steps = []

for i_episode in range(200):
    observation = env.reset()
    for t in range(1000):
        env.render()
#         print(observation)
        all_observations.append(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            steps.append(t+1)
#             print("Episode finished after {} timesteps".format(t+1))
            break

env.close()


In [5]:
len(all_observations)

4490

In [6]:
import pandas as pd

cols = ['pos', 'velocity', 'angle', 'angular velocity']
df = pd.DataFrame(data=all_observations, columns=cols)

In [7]:
df.describe()

,pos,velocity,angle,angular velocity
count,4490.000000,4490.000000,4490.000000,4490.000000
mean,-0.000524,0.013570,-0.004720,-0.024244
std,0.082894,0.535590,0.094283,0.781915
min,-0.530120,-2.154456,-0.209409,-2.590083
25%,-0.043038,-0.344192,-0.064430,-0.554512
50%,-0.002247,0.015890,-0.005197,-0.017951
75%,0.042638,0.372433,0.051348,0.461612
max,0.610815,1.796593,0.209186,2.664974


In [8]:
print('theoretical min:', env.observation_space.low)
print('theoretical max:', env.observation_space.high)

theoretical min: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
theoretical max: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


## Analyze game inputs on those observations

### Analyze firing rates

In [46]:
from neurosim.game_interface import GameInterface

config_fr = {
    "env": {
        "name": "CartPole-v1",
        "savemp4": 0,
        "render": 0,
        "observation_map": [
            {"type": "linear", "min": -0.7, "max": 0.7},
            {"type": "linear", "min": -2, "max": 2},
            {"type": "linear", "min": -0.2, "max": 0.2},
            {"type": "linear", "min": -2.5, "max": 2.5}
        ],
        "episodes": 21
    },
    "net": {
        "InputMaxRate": 50,
        "inputPop": "ES",
        "allpops": {
            "ES": 20
        }
    }
}

game = MockAIGame(env)
gi = GameInterface(game, config_fr)

In [47]:
all_firing_rates = []

for obs in all_observations:
    game.observations = [obs]
    all_firing_rates.append(gi.input_firing_rates()[:4])
    
print(len(all_firing_rates))

4490


In [48]:
df_fr = pd.DataFrame(data=all_firing_rates, columns=['fr_'+c for c in cols])

In [49]:
df_fr.describe()

,fr_pos,fr_velocity,fr_angle,fr_angular velocity
count,4490.000000,4490.000000,4490.000000,4490.000000
mean,24.981296,25.170055,24.413378,24.757554
std,2.960486,6.693191,11.758399,7.816748
min,6.067149,0.000000,0.000000,0.000000
25%,23.462918,20.697597,16.946197,19.454880
50%,24.919757,25.198627,24.350421,24.820487
75%,26.522775,29.655418,31.418556,29.616122
max,46.814825,47.457409,50.000000,50.000000


In [50]:
maxFiring = config['net']['InputMaxRate']
df_interval = pd.DataFrame(data=[[maxFiring/fr for fr in k]for k in all_firing_rates], columns=['int_'+c for c in cols])

/Users/danielhasegan/workspace/netpyne-project/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


In [51]:
df_interval.describe()

,int_pos,int_velocity,int_angle,int_angular velocity
count,4490.000000,4490.000000,4490.000000,4490.000000
mean,2.033834,inf,inf,inf
std,0.307321,NaN,NaN,NaN
min,1.068038,1.053576,1.000000,1.000000
25%,1.885172,1.686033,1.591416,1.688270
50%,2.006440,1.984235,2.053353,2.014465
75%,2.131022,2.415739,2.950515,2.570049
max,8.241103,inf,inf,inf


### Analyze Receptive Fields

In [112]:
all_smallest_distances = {}

In [128]:
from neurosim.game_interface import GameInterface

bin_size = 200

config_rf = {
    "env": {
        "name": "CartPole-v1",
        "savemp4": 0,
        "render": 0,
        "observation_map": [
            {"type": "rf_normal", "bins": bin_size, "mean": -0.00645789, "std": 0.06894682},
            {"type": "rf_normal", "bins": bin_size, "mean": -0.03634136, "std": 0.53989509},
            {"type": "rf_normal", "bins": bin_size, "mean": 0.00529333, "std": 0.09129902},
            {"type": "rf_normal", "bins": bin_size, "mean": 0.04752025, "std": 0.82070579}
        ],
        "episodes": 21
    },
    "net": {
        "InputMaxRate": 50,
        "inputPop": "ES",
        "allpops": {
            "ES": bin_size * 4
        }
    }
}

game = MockAIGame(env)
gi = GameInterface(game, config_rf)

In [129]:
all_firing_rates = []
all_firing_indices = []

for obs in all_observations:
    game.observations = [obs]
    all_firing_rates.append(gi.input_firing_rates())
    all_firing_indices.append((all_firing_rates[-1] > 0).nonzero()[0])
    
print(len(all_firing_rates))
print(len(all_firing_indices))

4490
4490


In [130]:
df_inds = pd.DataFrame(data=all_firing_indices, columns=['int_'+c for c in cols])

In [131]:
df_inds.describe()

,int_pos,int_velocity,int_angle,int_angular velocity
count,4490.000000,4490.000000,4490.000000,4490.000000
mean,103.760356,305.779955,493.290423,693.888864
std,56.869382,55.992045,58.580774,55.599701
min,0.000000,200.000000,401.000000,600.000000
25%,59.000000,256.000000,444.000000,646.000000
50%,104.000000,307.000000,490.000000,693.000000
75%,152.000000,355.000000,538.000000,738.000000
max,199.000000,399.000000,597.000000,799.000000


In [132]:
import numpy as np
MAX_VAL = 10000

all_smallest_distances[bin_size] = {}
for c_idx, c in enumerate(cols):
    c_min = float(df.describe().loc[['min']][c].to_numpy()[0])
    c_max = float(df.describe().loc[['max']][c].to_numpy()[0])
    print(c, c_min, c_max)
    observation = [0] * len(cols)
    c_maps = {}
    for v in np.arange(c_min, c_max, 0.00005):
        obs[c_idx] = v
        game.observations = [obs]
        f_r = gi.input_firing_rates()
        f_ind = (f_r > 0).nonzero()[0][c_idx]
#         print(f_ind)
        if f_ind not in c_maps:
            c_maps[f_ind] = [MAX_VAL, -MAX_VAL]
        if v < c_maps[f_ind][0]:
            c_maps[f_ind][0] = v
        if v > c_maps[f_ind][1]:
            c_maps[f_ind][1] = v
        
    smallest_rf = MAX_VAL
    for k,v in c_maps.items():
        print('    {}: {}\t\tsize_of_rf: {}'.format(k, v, round(v[1] - v[0], 7)))
        if smallest_rf > v[1] - v[0]:
            smallest_rf = v[1] - v[0]
            
    print('smallest_rf', smallest_rf)
    all_smallest_distances[bin_size][c] = smallest_rf
    print('')

pos -0.530119829813789 0.610815093724111
    0: [-0.530119829813789, -0.18406982981382713]		size_of_rf: 0.34605
    1: [-0.18401982981382714, -0.16686982981382903]		size_of_rf: 0.01715
    2: [-0.16681982981382903, -0.1561198298138302]		size_of_rf: 0.0107
    3: [-0.15606982981383022, -0.1480698298138311]		size_of_rf: 0.008
    4: [-0.1480198298138311, -0.1416198298138318]		size_of_rf: 0.0064
    5: [-0.14156982981383182, -0.1361698298138324]		size_of_rf: 0.0054
    6: [-0.13611982981383242, -0.13141982981383293]		size_of_rf: 0.0047
    7: [-0.13136982981383294, -0.1271698298138334]		size_of_rf: 0.0042
    8: [-0.1271198298138334, -0.12336982981383382]		size_of_rf: 0.00375
    9: [-0.12331982981383383, -0.1198698298138342]		size_of_rf: 0.00345
    10: [-0.11981982981383421, -0.11666982981383456]		size_of_rf: 0.00315
    11: [-0.11661982981383456, -0.11366982981383489]		size_of_rf: 0.00295
    12: [-0.1136198298138349, -0.1108698298138352]		size_of_rf: 0.00275
    13: [-0.11081982981383

    200: [-2.154455659493083, -1.427055659494778]		size_of_rf: 0.7274
    201: [-1.4270056594947782, -1.292355659495092]		size_of_rf: 0.13465
    202: [-1.2923056594950921, -1.2080056594952886]		size_of_rf: 0.0843
    203: [-1.2079556594952887, -1.145155659495435]		size_of_rf: 0.0628
    204: [-1.1451056594954352, -1.094555659495553]		size_of_rf: 0.05055
    205: [-1.0945056594955531, -1.0518056594956526]		size_of_rf: 0.0427
    206: [-1.0517556594956528, -1.0146056594957393]		size_of_rf: 0.03715
    207: [-1.0145556594957394, -0.9815556594958164]		size_of_rf: 0.033
    208: [-0.9815056594958165, -0.9517056594958859]		size_of_rf: 0.0298
    209: [-0.951655659495886, -0.9244056594959495]		size_of_rf: 0.02725
    210: [-0.9243556594959497, -0.8992056594960083]		size_of_rf: 0.02515
    211: [-0.8991556594960084, -0.8758056594960628]		size_of_rf: 0.02335
    212: [-0.8757556594960629, -0.8538056594961141]		size_of_rf: 0.02195
    213: [-0.8537556594961142, -0.8331556594961622]		size_of_rf:

    401: [-0.20940894387829306, -0.20710894387829332]		size_of_rf: 0.0023
    402: [-0.20705894387829332, -0.1928589438782949]		size_of_rf: 0.0142
    403: [-0.1928089438782949, -0.18225894387829605]		size_of_rf: 0.01055
    404: [-0.18220894387829606, -0.173658943878297]		size_of_rf: 0.00855
    405: [-0.173608943878297, -0.1664589438782978]		size_of_rf: 0.00715
    406: [-0.1664089438782978, -0.1601589438782985]		size_of_rf: 0.00625
    407: [-0.1601089438782985, -0.1545589438782991]		size_of_rf: 0.00555
    408: [-0.1545089438782991, -0.14950894387829966]		size_of_rf: 0.005
    409: [-0.14945894387829967, -0.14490894387830017]		size_of_rf: 0.00455
    410: [-0.14485894387830017, -0.14065894387830064]		size_of_rf: 0.0042
    411: [-0.14060894387830064, -0.13665894387830108]		size_of_rf: 0.00395
    412: [-0.13660894387830108, -0.13295894387830148]		size_of_rf: 0.00365
    413: [-0.1329089438783015, -0.12945894387830187]		size_of_rf: 0.00345
    414: [-0.12940894387830187, -0.12615894

    600: [-2.5900831285195154, -2.0664831285207357]		size_of_rf: 0.5236
    601: [-2.066433128520736, -1.8617331285212129]		size_of_rf: 0.2047
    602: [-1.861683128521213, -1.7335331285215116]		size_of_rf: 0.12815
    603: [-1.7334831285215118, -1.6380331285217342]		size_of_rf: 0.09545
    604: [-1.6379831285217343, -1.5610831285219136]		size_of_rf: 0.0769
    605: [-1.5610331285219137, -1.496083128522065]		size_of_rf: 0.06495
    606: [-1.4960331285220652, -1.4395331285221968]		size_of_rf: 0.0565
    607: [-1.439483128522197, -1.389283128522314]		size_of_rf: 0.0502
    608: [-1.389233128522314, -1.3439331285224196]		size_of_rf: 0.0453
    609: [-1.3438831285224198, -1.3024331285225164]		size_of_rf: 0.04145
    610: [-1.3023831285225165, -1.2641331285226056]		size_of_rf: 0.03825
    611: [-1.2640831285226057, -1.2285331285226886]		size_of_rf: 0.03555
    612: [-1.2284831285226887, -1.1951331285227664]		size_of_rf: 0.03335
    613: [-1.1950831285227665, -1.1636831285228397]		size_of_rf

In [133]:

all_smallest_distances

{20: {'pos': 0.008599999999999053,
  'velocity': 0.06779999999984199,
  'angle': 0.011399999999998744,
  'angular velocity': 0.10304999999975983},
 10: {'pos': 0.017399999999998084,
  'velocity': 0.1366999999996814,
  'angle': 0.02304999999999746,
  'angular velocity': 0.2078499999995156},
 40: {'pos': 0.004249999999999532,
  'velocity': 0.033799999999921226,
  'angle': 0.005649999999999378,
  'angular velocity': 0.05139999999988021},
 200: {'pos': 0.0007999999999999119,
  'velocity': 0.006699999999984385,
  'angle': 0.0010499999999998844,
  'angular velocity': 0.010199999999976228}}

## Analyze steps per episode

In [15]:
df_steps = pd.DataFrame(steps, columns=['steps_per_episode'])
df_steps.describe()

,steps_per_episode
count,200.000000
mean,21.790000
std,11.855745
min,9.000000
25%,14.000000
50%,18.000000
75%,26.000000
max,84.000000


Metric to evalute: rolling median should be higher than 17

In [3]:
import csv
import pandas as pd
items = []
with open('/Users/danielhasegan/workspace/netpyne-project/results/20210617/ActionsPerEpisode.txt') as f:
    for row in csv.reader(f, delimiter='\t'):
        items.append(int(float(row[1])))


In [12]:

STEP = int(len(items) / 4)
for i in range(int(len(items) / STEP)):
    sub_items = items[STEP * i: STEP * (i+1)]
    print(pd.DataFrame(sub_items, columns=['st_per_ep']).describe())

        st_per_ep
count  209.000000
mean    24.148325
std     12.312127
min      9.000000
25%     15.000000
50%     21.000000
75%     29.000000
max     78.000000
        st_per_ep
count  209.000000
mean    22.784689
std      9.901270
min      9.000000
25%     16.000000
50%     21.000000
75%     28.000000
max     67.000000
        st_per_ep
count  209.000000
mean    22.712919
std     11.467088
min      9.000000
25%     14.000000
50%     19.000000
75%     28.000000
max     63.000000
        st_per_ep
count  209.000000
mean    23.909091
std     11.463419
min      9.000000
25%     15.000000
50%     21.000000
75%     30.000000
max     72.000000


In [7]:
len(items) / 4

209.0